# The smell of classes

## The "data bundle" smell

In [34]:
def momentum(mass, velocity):
    return mass * velocity

def energy(mass, velocity):
    return 0.5 * mass * velocity ** 2

def update_position(velocity, position, dt):
    return position + velocity * dt

In [35]:
# Naive
mass1 = 10.0
velocity1 = 0.9
position1 = 0.0

mass2 = 12.0
velocity2 = 0.1
position2 = -23.0

print(momentum(mass1, velocity1))
print(momentum(mass2, velocity2))
print(momentum(mass1, velocity2))  # ??

9.0
1.2000000000000002
1.0


We have three parameters that will be sent to these functions over and over again: `mass`, `velocity`, and `position`.

Moreover, the parameters cannot be mixed up (e.g. the velocity of one particle with the mass of another).

In [36]:
masses = [10.0, 12.0]
velocities = [0.9, 0.1]
positions = [0.0, -23.0]

print(momentum(masses[0], velocities[0]))
print(momentum(masses[1], velocities[1]))

9.0
1.2000000000000002


In [37]:
particle1 = {'mass': 10.0, 'velocity': 0.9, 'position': 0.0}
particle2 = {'mass': 12.0, 'velocity': 0.1, 'position': -23.0}

print(momentum(particle1['mass'], particle1['velocity']))
print(momentum(particle2['mass'], particle2['velocity']))


9.0
1.2000000000000002


All of the functions above can be rewritten as a function of this particle "object", eliminating the bookkeeping for the individual parameters.

In [38]:
def momentum(particle):
    return particle['mass'] * particle['velocity']

print(momentum(particle1))
print(momentum(particle2))


9.0
1.2000000000000002


In [41]:
def init_particle(mass, velocity, position=0.0):
    self = {
        'mass': mass,
        'velocity': velocity,
        'position': position,
    }
    return self

particle1 = init_particle(10.0, 0.9)
particle2 = init_particle(12.0, 0.1, -23.0)
print(momentum(particle1))
print(momentum(particle2))


9.0
1.2000000000000002


`particle1` and `particle2` are called "instances" of the particle "class".

Python classes are a way to formalize this pattern.

## Introducing classes as a data bundle template

In [61]:
class Particle:
    def __init__(self, mass, velocity, position=0.0):
        self.mass = mass
        self.velocity = velocity
        self.position = position

particle1 = Particle(10.0, 0.9)
particle2 = Particle(12.0, 0.1, -23.0)

In [44]:
particle1.velocity

0.9

In [45]:
particle2.mass

12.0

In [46]:
particle1.__dict__

{'mass': 10.0, 'position': 0.0, 'velocity': 0.9}

## Exercise 1: Simple parameters class

In [94]:
# ??? Something with a 3D coord vector; Cube3D class, with rotate etc. methods?

In [96]:
import numpy as np

position = np.array([-1.0, 1.0, 3.0])
size = 0.3
rotation = np.array(
    [[1.0, 0.0, 0.0],
     [0.0, 1.0, 0.0],
     [0.0, 0.0, 1.0]
    ]
)

In [98]:
# ??? A markov model?

In [ ]:
# Probability of first state
prob_first = np.array([0.1, 0.7, 0.2])
prob_next = np.array([
        [0.25, 0.25, 0.5],
        [0.3, 0.2, 0.5],
        [0.7, 0.1, 0.2],
    ])

def predict_first(prob_first):
    np.random.binomial

def predict_next(current_state, prob_next):
    pass

def predict(n, prob_first, prob_next, current_state=None):
    pass

# THIS ONE as additional class method in another exercise: from_observations
def learn(states):
    pass


## Class methods

In [21]:
def momentum(particle):
    return particle.mass * particle.velocity

print(momentum(particle1))
print(momentum(particle2))

9.0
1.2000000000000002


In [30]:
class Particle:
    def __init__(self, mass, velocity, position=0.0):
        self.mass = mass
        self.velocity = velocity
        self.position = position

    def momentum(self):
        return self.mass * self.velocity
    
    def energy(self):
        return 0.5 * self.mass * self.velocity ** 2

particle1 = Particle(10.0, 0.9, 0.0)
print(particle1.momentum())

9.0


We have been using class instances and methods all along...

In [54]:
s = 'A scanner Darkly'
s.capitalize()

'A scanner darkly'

In [92]:
x = set(['apple', 'banana', 'apple', 'pineapple'])
x

{'apple', 'banana', 'pineapple'}

In [91]:
x.union(['banana', 'kiwi'])

{'apple', 'banana', 'kiwi', 'pineapple'}

## Giving the class more responsibilities: serialization

In [97]:
# ??? Add something easier before this
# ??? store initial coordinates, add a "reset" method

# ??? replace particle with something a bit more interesting as a probability distribution

In [63]:
import json

class Particle:
    def __init__(self, mass, velocity, position=0.0):
        self.mass = mass
        self.velocity = velocity
        self.position = position

    def momentum(self):
        return self.mass * self.velocity
    
    def energy(self):
        return 0.5 * self.mass * self.velocity ** 2
    
    # --- Serialization

    def to_json(self):
        params = {
            'mass': self.mass,
            'velocity': self.velocity,
            'position': self.position,
        }
        return json.dumps(params)

In [82]:
particle = Particle(13.0, 0.9, position=3.2)
json_str = particle.to_json()

with open('model_v1.json', 'wt') as f:
    f.write(json_str)

In [83]:
!cat model_v1.json

{"velocity": 0.9, "mass": 13.0, "position": 3.2}

In [87]:
class Particle:
    def __init__(self, mass, velocity, position=0.0):
        self.mass = mass
        self.velocity = velocity
        self.position = position

    def momentum(self):
        return self.mass * self.velocity
    
    def energy(self):
        return 0.5 * self.mass * self.velocity ** 2
    
    # --- Serialization

    def to_json(self):
        params = {
            'mass': self.mass,
            'velocity': self.velocity,
            'position': self.position,
        }
        return json.dumps(params)
    
    @classmethod
    def from_json(cls, json_str):
        params = json.loads(json_str)
        return cls(**params)

In [85]:
with open('model_v1.json', 'rt') as f:
    json_str = f.read()
    
particle = Particle.from_json(json_str)

In [86]:
particle.mass

13.0

## Exercise 2: Add serialization methods to the class of the previous exercise

## Real-life examples

https://bitbucket.org/pberkes/content_style_model/src/master/model/objects.py

https://bitbucket.org/pberkes/brandeis-general-repository/src/master/trunk/projects/overcomplete/gaussmix/objects.py

## Exercise 3: Add "from_observations" factory method